The functools.cache decorator implements memoization: 5 an
optimization technique that works by saving the results of previous
invocations of an expensive function, avoiding repeat computations on
previously used arguments.

Example 9-17. The very costly recursive way to compute the nth number in
the Fibonacci series

In [1]:
from clockdeco import clock


@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000063s] fibonacci(0) -> 0
[0.00000081s] fibonacci(1) -> 1
[0.00010406s] fibonacci(2) -> 1
[0.00000060s] fibonacci(1) -> 1
[0.00000051s] fibonacci(0) -> 0
[0.00000064s] fibonacci(1) -> 1
[0.00006002s] fibonacci(2) -> 1
[0.00011270s] fibonacci(3) -> 2
[0.00027057s] fibonacci(4) -> 3
[0.00000044s] fibonacci(1) -> 1
[0.00000041s] fibonacci(0) -> 0
[0.00000067s] fibonacci(1) -> 1
[0.00006780s] fibonacci(2) -> 1
[0.00011756s] fibonacci(3) -> 2
[0.00000044s] fibonacci(0) -> 0
[0.00000053s] fibonacci(1) -> 1
[0.00005049s] fibonacci(2) -> 1
[0.00000047s] fibonacci(1) -> 1
[0.00000047s] fibonacci(0) -> 0
[0.00000056s] fibonacci(1) -> 1
[0.00005110s] fibonacci(2) -> 1
[0.00009983s] fibonacci(3) -> 2
[0.00019936s] fibonacci(4) -> 3
[0.00038036s] fibonacci(5) -> 5
[0.00070237s] fibonacci(6) -> 8
8


Example 9-18. Faster implementation using caching

In [5]:
import functools

from clockdeco import clock

@functools.cache
@clock
def fast_fibonacci(n):
    if n < 2:
        return n
    return fast_fibonacci(n -2) + fast_fibonacci(n -1)

if __name__ == '__main__':
    print(fast_fibonacci(6))

[0.00000070s] fast_fibonacci(0) -> 0
[0.00000094s] fast_fibonacci(1) -> 1
[0.00013731s] fast_fibonacci(2) -> 1
[0.00000111s] fast_fibonacci(3) -> 2
[0.00019896s] fast_fibonacci(4) -> 3
[0.00000100s] fast_fibonacci(5) -> 5
[0.00025733s] fast_fibonacci(6) -> 8
8


### Using lru_cache
The functools.cache decorator is actually a simple wrapper around
the older functools.lru_cache function, which is more flexible and
compatible with Python 3.8 and earlier versions.

Single Dispatch Generic Functions

Imagine we are creating a tool to debug web applications. We want to
generate HTML displays for different types of Python objects.

In [6]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [7]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [8]:
htmlize('my name is Jalil.\n a game')

'<pre>&#x27;my name is Jalil.\\n a game&#x27;</pre>'

Example 9-20. @singledispatch creates a custom
@htmlize.register to bundle several functions into a generic
function

In [10]:
from functools import singledispatch
from collections import abc
import fractions 
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'
@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'
@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'
@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'
@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'
@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'
@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'
